# Deploy IDP with Amazon Bedrock MCP Server to Amazon Bedrock AgentCore Runtime

This notebook deploys the IDP with Amazon Bedrock document attribute extraction system as an MCP server using Amazon Bedrock AgentCore Runtime.

## Overview

The IDP with Amazon Bedrock MCP Server exposes the existing IDP functionality (document attribute extraction) as MCP tools that can be used by any MCP-compatible client or AI agent.

### Key Features
- **Reuses existing infrastructure**: Leverages deployed Step Functions and S3 bucket
- **Existing Cognito integration**: Uses the current Cognito user pool for authentication
- **MCP-compliant tools**: Provides standardized MCP interface for document processing
- **Fully managed**: Hosted on AgentCore Runtime with automatic scaling

### MCP Tools Provided
1. **`extract_document_attributes`**: Main tool for document attribute extraction
2. **`get_extraction_status`**: Check status of long-running extractions
3. **`list_supported_models`**: Get available Bedrock models
4. **`get_bucket_info`**: Information about S3 bucket and supported formats

## Prerequisites

- IDP with Amazon Bedrock project deployed with Step Functions and Cognito
- AWS credentials configured
- Python 3.10+
- Docker running (for AgentCore Runtime deployment)

## Step 1: Install Dependencies

In [2]:
# Install required packages
!uv pip install bedrock-agentcore-starter-toolkit boto3 mcp fastmcp

Using Python 3.13.3 environment at: /Users/egorkr/Projects/IDP/.venv
Audited 4 packages in 48ms


## Step 2: Import Dependencies and Setup

In [3]:
import os
import time
from boto3.session import Session

# Import our utility functions
from utils import (
    get_existing_cognito_config,
    get_existing_infrastructure_config,
    create_mcp_user_in_existing_pool,
    create_agentcore_role,
    store_mcp_configuration,
)

# Get AWS region
boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

Using AWS region: us-east-1


## Step 3: Verify Existing Infrastructure

First, let's verify that the IDP project is deployed and we can access the existing resources.

In [4]:
print("🔍 Checking existing IDP infrastructure...")
print("=" * 60)

# Get existing Cognito configuration
cognito_config = get_existing_cognito_config()
if not cognito_config:
    raise Exception("Could not find existing Cognito configuration. Make sure IDP project is deployed.")

print("\n" + "=" * 60)

# Get existing infrastructure configuration
infra_config = get_existing_infrastructure_config()
if not infra_config:
    raise Exception("Could not find existing infrastructure. Make sure IDP project is deployed.")

print("\n✅ All existing infrastructure verified!")

🔍 Checking existing IDP infrastructure...
✅ Found existing Cognito configuration:
   User Pool ID: us-east-1_q5PSP2NkZ
   Client ID: 3g145h569975hdcg68onjeo42h
   Domain: idp-bedrock-471112811980.auth.us-east-1.amazoncognito.com
   Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_q5PSP2NkZ/.well-known/openid-configuration

✅ Found Step Functions: arn:aws:states:us-east-1:471112811980:stateMachine:idp-bedrock-StepFunctions
✅ Found S3 bucket: idp-bedrock-data-471112811980

✅ All existing infrastructure verified!


## Step 4: Use Existing Cognito User

We'll use an existing user from the Cognito user pool for MCP access.

In [5]:
print("👤 Using existing user from Cognito pool...")
print("=" * 60)

# Use existing user (from config.yml: egorkr@amazon.co.uk)
# You'll be prompted for the password
mcp_user_config = create_mcp_user_in_existing_pool(
    cognito_config=cognito_config,
    username="egorkr@amazon.co.uk",  # This should match the user from config.yml
    password=None,  # Will prompt for password
)

if not mcp_user_config:
    raise Exception("Failed to authenticate existing user")

print("\n✅ User authenticated successfully!")
print(f"Username: {mcp_user_config['username']}")

👤 Using existing user from Cognito pool...
⚠️  Password required for user: egorkr@amazon.co.uk
Please provide the password for this user, or set a new one in Cognito console
✅ Successfully authenticated user: egorkr@amazon.co.uk
✅ Generated bearer token for MCP access

✅ User authenticated successfully!
Username: egorkr@amazon.co.uk


## Step 5: Create IAM Role for AgentCore Runtime

Create an IAM role with permissions to access the IDP resources (Step Functions, S3, Bedrock).

In [6]:
print("🔐 Creating IAM role for AgentCore Runtime...")
print("=" * 60)

# Create IAM role
agentcore_iam_role = create_agentcore_role(agent_name="IDPagent")

print("\n✅ IAM role created successfully!")
print(f"Role ARN: {agentcore_iam_role['Role']['Arn']}")

🔐 Creating IAM role for AgentCore Runtime...
ℹ️  Role agentcore-IDPagent-role already exists, recreating...
✅ Recreated IAM role: agentcore-IDPagent-role
✅ Attached policy to role: agentcore-IDPagent-role

✅ IAM role created successfully!
Role ARN: arn:aws:iam::471112811980:role/agentcore-IDPagent-role


## Step 6: Configure AgentCore Runtime Deployment

Configure the AgentCore Runtime with our MCP server and authentication. The MCP server will automatically discover the infrastructure configuration.

In [7]:
from bedrock_agentcore_starter_toolkit import Runtime

print("⚙️  Configuring AgentCore Runtime deployment...")
print("=" * 60)

# Verify required files exist
required_files = ["mcp_server.py", "requirements.txt"]
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("✅ All required files found")

# Initialize AgentCore Runtime
agentcore_runtime = Runtime()

# Configure authentication with existing Cognito
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config["client_id"]],
        "discoveryUrl": cognito_config["discovery_url"],
    }
}

print("🔧 Configuring runtime...")
print("   Infrastructure will be discovered automatically by the MCP server")
print(f"   Expected State Machine: {infra_config['state_machine_arn']}")
print(f"   Expected S3 Bucket: {infra_config['bucket_name']}")

response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    execution_role=agentcore_iam_role["Role"]["Arn"],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="IDPagent",
)

print("✅ AgentCore Runtime configured successfully!")

Entrypoint parsed: file=/Users/egorkr/Projects/IDP/mcp_deploy/IDP_mcp_server/mcp_server.py, bedrock_agentcore_name=mcp_server
Configuring BedrockAgentCore agent: IDPagent


⚙️  Configuring AgentCore Runtime deployment...
✅ All required files found
🔧 Configuring runtime...
   Infrastructure will be discovered automatically by the MCP server
   Expected State Machine: arn:aws:states:us-east-1:471112811980:stateMachine:idp-bedrock-StepFunctions
   Expected S3 Bucket: idp-bedrock-data-471112811980


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

Generated Dockerfile: /Users/egorkr/Projects/IDP/mcp_deploy/IDP_mcp_server/Dockerfile
Generated .dockerignore: /Users/egorkr/Projects/IDP/mcp_deploy/IDP_mcp_server/.dockerignore
Keeping 'IDPagent' as default agent
Bedrock AgentCore configured: /Users/egorkr/Projects/IDP/mcp_deploy/IDP_mcp_server/.bedrock_agentcore.yaml


✅ AgentCore Runtime configured successfully!


## Step 7: Launch MCP Server to AgentCore Runtime

Deploy the MCP server to AWS. This will create the ECR repository, build the Docker image, and deploy to AgentCore Runtime.

In [8]:
print("🚀 Launching IDP with Amazon Bedrock MCP Server to AgentCore Runtime...")
print("=" * 60)
print("⏳ This may take several minutes...")

# Launch the MCP server
launch_result = agentcore_runtime.launch()

print("\n✅ Launch completed successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

# Store the results for later use
agent_arn = launch_result.agent_arn
agent_id = launch_result.agent_id

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'IDPagent' to account 471112811980 (us-east-1)
Starting CodeBuild ARM64 deployment for agent 'IDPagent' to account 471112811980 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: IDPagent


🚀 Launching IDP with Amazon Bedrock MCP Server to AgentCore Runtime...
⏳ This may take several minutes...


✅ ECR repository available: 471112811980.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-IDPagent
Using execution role from config: arn:aws:iam::471112811980:role/agentcore-IDPagent-role


✅ Reusing existing ECR repository: 471112811980.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-IDPagent


✅ Execution role validation passed: arn:aws:iam::471112811980:role/agentcore-IDPagent-role
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: IDPagent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-167ce93a9b
Reusing existing CodeBuild execution role: arn:aws:iam::471112811980:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-167ce93a9b
Using .dockerignore with 44 patterns
Uploaded source to S3: IDPagent/20250802-231035.zip
Updated CodeBuild project: bedrock-agentcore-IDPagent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 5.1s
🔄 PROVISIONING started (total: 5s)
✅ PROVISIONING completed in 10.3s
🔄 PRE_BUILD started (total: 16s)
✅ PRE_BUILD completed in 10.3s
🔄 BUILD started (total: 26s)
✅ BUILD completed in 36.0s
🔄 POST_BUILD started (total: 62s)
✅ POST_BUILD completed in 5.2s
🔄 COMPLETED started (total: 67s)
✅ COMPLE


✅ Launch completed successfully!
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:471112811980:runtime/IDPagent-1fcZqaBeRk
Agent ID: IDPagent-1fcZqaBeRk


## Step 8: Wait for Deployment to Complete

Monitor the deployment status and wait for it to be ready.

In [9]:
print("⏳ Checking AgentCore Runtime deployment status...")
print("=" * 60)

# Check initial status
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]
print(f"Initial status: {status}")

# Wait for completion
end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(30)  # Check every 30 seconds
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

if status == "READY":
    print("\n🎉 AgentCore Runtime is READY!")
    print("✅ IDP with Amazon Bedrock MCP Server deployed successfully!")
else:
    print(f"\n⚠️  AgentCore Runtime status: {status}")
    if status in ["CREATE_FAILED", "UPDATE_FAILED"]:
        print("❌ Deployment failed. Check CloudWatch logs for details.")

print(f"\nFinal status: {status}")

⏳ Checking AgentCore Runtime deployment status...


Retrieved Bedrock AgentCore status for: IDPagent


Initial status: READY

🎉 AgentCore Runtime is READY!
✅ IDP with Amazon Bedrock MCP Server deployed successfully!

Final status: READY


## Step 9: Store Configuration for Remote Access

Store the deployment configuration in AWS Systems Manager and Secrets Manager for easy access by MCP clients.

In [10]:
print("💾 Storing MCP configuration for remote access...")
print("=" * 60)

# Store configuration
config_stored = store_mcp_configuration(
    agent_arn=agent_arn, cognito_config=cognito_config, mcp_user_config=mcp_user_config
)

if config_stored:
    print("\n✅ Configuration stored successfully!")
    print("\n📋 Deployment Summary:")
    print("=" * 40)
    print(f"Agent ARN: {agent_arn}")
    print(f"Agent ID: {agent_id}")
    print(f"MCP User: {mcp_user_config['username']}")
    print(f"Cognito Client ID: {cognito_config['client_id']}")
    print(f"State Machine: {infra_config['state_machine_arn']}")
    print(f"S3 Bucket: {infra_config['bucket_name']}")
    print("\n🔗 Access Information:")
    print("Parameter Store: /IDP-mcp/runtime/agent_arn")
    print("Secrets Manager: IDP-mcp/cognito/credentials")
else:
    print("❌ Failed to store configuration")

💾 Storing MCP configuration for remote access...
✅ Agent ARN stored in Parameter Store
✅ MCP credentials updated in Secrets Manager

✅ Configuration stored successfully!

📋 Deployment Summary:
Agent ARN: arn:aws:bedrock-agentcore:us-east-1:471112811980:runtime/IDPagent-1fcZqaBeRk
Agent ID: IDPagent-1fcZqaBeRk
MCP User: egorkr@amazon.co.uk
Cognito Client ID: 3g145h569975hdcg68onjeo42h
State Machine: arn:aws:states:us-east-1:471112811980:stateMachine:idp-bedrock-StepFunctions
S3 Bucket: idp-bedrock-data-471112811980

🔗 Access Information:
Parameter Store: /IDP-mcp/runtime/agent_arn
Secrets Manager: IDP-mcp/cognito/credentials


## Step 10: Create Remote Testing Client

Create a client script that can connect to the deployed MCP server for testing.

## Step 11: Test the Deployed MCP Server

Let's test our deployed MCP server to make sure everything is working correctly.

In [13]:
print("🧪 Testing the deployed IDP with Amazon Bedrock MCP Server...")
print("=" * 60)

# Run the remote test client
!python test_client_remote.py

🧪 Testing the deployed IDP with Amazon Bedrock MCP Server...
🔍 Testing deployed IDP with Amazon Bedrock MCP Server in {region}...
✅ Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-east-1:471112811980:runtime/IDPagent-1fcZqaBeRk
✅ Retrieved bearer token

🔗 Connecting to: https://bedrock-agentcore.us-east-1.amazonaws.com/runtimes/arn%3Aaws%3Abedrock-agentcore%3Aus-east-1%3A471112811980%3Aruntime%2FIDPagent-1fcZqaBeRk/invocations?qualifier=DEFAULT

🔄 Initializing MCP session...
✅ MCP session initialized

🔄 Listing available tools...

📋 Available IDP with Amazon Bedrock MCP Tools:
🔧 extract_document_attributes
   Description: 
Extract custom attributes from documents using Amazon Bedrock and AWS document processing services.

This tool processes documents (text, PDF, images, Office files) and extracts specified attributes
using large language models. It supports various parsing modes and can handle batch processing.

Args:
    documents: List of document paths/keys in S3 (e.g., ["origina

In [16]:
!python test_with_demo_document.py

🚀 IDP with Amazon Bedrock MCP Server Demo Document Test
This script tests document attribute extraction using email_1.txt

🧪 Testing IDP with Amazon Bedrock MCP Server with demo document...
✅ Retrieved Agent ARN: arn:aws:bedrock-agentcore:us-east-1:471112811980:runtime/IDPagent-1fcZqaBeRk
✅ Retrieved bearer token

🔗 Connecting to MCP server...
🔄 Initializing MCP session...
✅ MCP session initialized

📋 Testing with demo document: originals/email_1.txt
➡️  Extracting attributes from email_1.txt...
✅ Extraction completed!

📊 Results for 1 document(s):
----------------------------------------
📄 Document: processed/email_1.txt
   Extracted attributes:
   • customer_name: John Smith
   • sentiment: negative
   • urgency: medium
   • main_topic: checking shipment status


🔧 Testing other MCP tools:
➡️  Getting supported models...
✅ Found 6 supported models
   Default model: us.anthropic.claude-3-haiku-20240307-v1:0

➡️  Getting bucket information...
✅ S3 Bucket: idp-bedrock-data-471112811980


## 🎉 Deployment Complete!

Your IDP with Amazon Bedrock MCP Server has been successfully deployed to Amazon Bedrock AgentCore Runtime!

### What's Available

✅ **MCP Server**: Deployed and running on AgentCore Runtime  
✅ **Authentication**: Integrated with existing Cognito user pool  
✅ **Tools**: 4 MCP tools for document attribute extraction  
✅ **Infrastructure**: Reuses existing Step Functions and S3 bucket  
✅ **Testing**: Remote client for validation  

### MCP Tools Available

1. **`extract_document_attributes`**: Extract custom attributes from documents
2. **`get_extraction_status`**: Check status of extraction operations
3. **`list_supported_models`**: Get available Bedrock models
4. **`get_bucket_info`**: S3 bucket information and supported formats

### Next Steps

- Use `test_client_remote.py` to test the MCP server
- Integrate with MCP-compatible clients (Claude, other AI agents)
- Upload documents to the S3 bucket for processing
- Monitor usage through CloudWatch logs

### Configuration Storage

- **Agent ARN**: Stored in Parameter Store at `/IDP-mcp/runtime/agent_arn`
- **Credentials**: Stored in Secrets Manager at `IDP-mcp/cognito/credentials`

The MCP server is now ready for production use! 🚀